<a href="https://colab.research.google.com/github/TunaInABottle/data_mining_2022/blob/main/ModifiedVersion_FriendsAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/TunaInABottle/data_mining_2022.git

fatal: destination path 'data_mining_2022' already exists and is not an empty directory.


In [ ]:
import random
import csv
import time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [ ]:
""" Import our own datasets   
"""
um= pd.read_csv('/content/data_mining_2022/data/size_500_1/utility_matrix.csv',index_col=0)
um.shape

(2000, 500)

In [ ]:
#collaborative filtreing with query to query similarity 
def predict_queries(utility_matrix,q_sim_matrix):
    utility_matrix_copy=utility_matrix.copy()
    utility_matrix_copy=utility_matrix_copy - utility_matrix_copy.mean()
    utility_matrix_copy=utility_matrix_copy.fillna(0)
    #get utility matrix dimensions
    rows = len(utility_matrix.axes[0])
    cols = len(utility_matrix.axes[1])
    for k in range(rows):
        for l in range(cols):
            if utility_matrix_copy.iloc[k,l]==0:
                weighted_score=np.dot(utility_matrix_copy.iloc[k],q_sim_matrix[l])
                weights_used=np.dot(q_sim_matrix[l],utility_matrix.iloc[k].notna())
                utility_matrix_copy.iloc[k,l]= round(weighted_score/weights_used,2)
            utility_matrix_copy.iloc[k,l]=min(max(utility_matrix_copy.iloc[k,l] + utility_matrix.mean()[l],0),100)
                
    return utility_matrix_copy

In [ ]:
#collaborative filtreing with user to user similarity 
def predict_users(utility_matrix,u_sim_matrix):
    utility_matrix_copy=utility_matrix.copy()
    utility_matrix_copy=utility_matrix_copy - utility_matrix_copy.mean()
    utility_matrix_copy=utility_matrix_copy.fillna(0)

    #get utility matrix dimensions
    rows = len(utility_matrix.axes[0])
    cols = len(utility_matrix.axes[1])
    columns=utility_matrix.axes[1]
    for k in range(rows):
        for l in range(cols):
            if utility_matrix_copy.iloc[k,l]==0:
                weighted_score=np.dot(utility_matrix.fillna(0)[columns[l]],u_sim_matrix[k])
                weights_used=np.dot(u_sim_matrix[k],utility_matrix[columns[l]].notna())
                utility_matrix_copy.iloc[k,l]=round(weighted_score/weights_used,2)
            utility_matrix_copy.iloc[k,l]=min(max(utility_matrix_copy.iloc[k,l] + utility_matrix.mean()[l],0),100)
    return utility_matrix_copy


In [ ]:
import math
import random
import statistics
import seaborn as sns
import matplotlib


def test_err (Train_matrix, L, queries_similarity=True, tpred=20, predi=70):
  #Root-mean-square error
  if queries_similarity:
    #Compute queries similarity matrix
    q_sim_matrix=cosine_similarity(Train_matrix.fillna(0).T) 
    pred=predict_queries(Train_matrix,q_sim_matrix)  
  else:    
    #Compute user similarity matrix
    #u_sim_matrix=cosine_similarity(Train_matrix.fillna(0))
    u_sim_matrix=pd.DataFrame(np.array(cosine_similarity(utility_matrix.fillna(0)))[0:100,:])
    pred=predict_queries(Train_matrix,u_sim_matrix)
  s=0
  TP=0
  TP_FN=0
  Recomm_queries=0
  if len(L)>0 :
    for i in L:
      s+= (i[0] - pred.iloc[i[1], i[2]])**2
      if abs(i[0] - pred.iloc[i[1], i[2]])<= tpred:
        TP_FN+=1
        if pred.iloc[i[1], i[2]] >= predi:
          TP+=1
      if pred.iloc[i[1], i[2]] >= predi:
        Recomm_queries+=1
    s= math.sqrt(s/len(L))
    
  return( [s, TP, TP_FN, Recomm_queries] )


def create_folds (utility_matrix, n, n_folds):
  """ Create X_train, X_test, in order to be able to train the algorithm on (n_folds)-1 folds,
      and test it on the remaining one, n_folds times
      :param utility_matrix: the utility matrix
      :param n: number of the iteration (the execution of the hole algorithm of the train/test)
      :param n_folds: number of folds
      :returns new_df: X_train
      :returns l: list of the values, and their indexes, left for X_test
  """
  new_df=utility_matrix.copy()
  #new_df.drop(columns='user_id', inplace=True)
  l=[]
  for i in range (len(new_df)//n_folds) :
    for j in range (len(new_df.columns)//n_folds):
      j=j+n*n_folds 
      for k in range (n_folds):
        u=(i+k*n_folds) % len(new_df)
        v=(j+k*n_folds) % len(new_df.columns)
        if not(pd.isna(new_df.iat[u, v])):
          l.append([new_df.iloc[u, v], u, v])
          new_df.iloc[u, v] = np.nan

  #new_df.insert(0, 'user_id', utility_matrix['user_id'])
  return(new_df, l)

from operator import add 

def Precision (TP, pred):
  return(TP/pred)
def Recall (TP,TP_FN):
  return(TP/TP_FN)
def F1_score (precision, recall):
  return(2*precision*recall/(precision+recall))

def cross_validation(utility_matrix, n_folds, queries_similarity=True):
  """ Evaluate the model using cross validation
      :param utility_matrix: the utility matrix
      :param asked_queries: full description of the queries
      :param query_combinations: a table containing all keys and values a query can have
      :param n_folds: number of folds
      :returns: the computed error 
  """  
  F=[0, 0, 0, 0]
  for i in range (n_folds):
    # Split the data into training and test sets
    new_df, l= create_folds (utility_matrix, i, n_folds)
    # Add the error of iteration i
    T= test_err (new_df, l, queries_similarity)
    F=list(map(add, T, F))
  F[0]=F[0]/n_folds
  if F[1]!=0:
    p = Precision (F[1], F[3])
    r = Recall(F[1], F[2])
    F[3] = F1_score(p,r)
    F[1] = p
    F[2] = r
  return(F)

In [ ]:
p="100_1"
utility_matrix= pd.read_csv('/content/data_mining_2022/data/size_' + p + '/utility_matrix.csv',index_col=0)
cross_validation(utility_matrix, 10, False)


KeyboardInterrupt: ignored

In [ ]:
a="500_1"

utility_matrix= pd.read_csv('/content/data_mining_2022/data/size_' + a + '/utility_matrix.csv',index_col=0)
queries_sim= cross_validation(utility_matrix, 10)
#users_sim= cross_validation(utility_matrix, 10, False)

In [ ]:
queries_sim

[[47.610416844749054, 0, 23, 67],
 [43.42540116647213, 0.047619047619047616, 0.15, 0.07228915662650602],
 [49.69434976906684, 0.03296703296703297, 0.25, 0.05825242718446602],
 [51.959310452079194,
  0.11130465075770772,
  0.5289735099337748,
  0.1839113541516765],
 [49.40779793121193,
  0.12296374146085129,
  0.536697247706422,
  0.20008550662676358],
 [52.83008854227135,
  0.07118385991625428,
  0.4087431693989071,
  0.12125141838223374]]

In [ ]:
users_sim

[[48.531602538491015, 0, 26, 68],
 [43.07964603911692,
  0.047619047619047616,
  0.13043478260869565,
  0.06976744186046512],
 [51.196519298119085, 0.03296703296703297, 0.25, 0.05825242718446602],
 [52.25593042629144,
  0.1075752334832238,
  0.5329905741216795,
  0.17901856382213266],
 [50.02016909223557,
  0.12199791159067178,
  0.5330798479087453,
  0.1985554454043337],
 [53.6226816648983,
  0.06560747663551401,
  0.43333333333333335,
  0.11396103896103896]]